# Automatische Übersetzung mit ChatGPT in einfache Sprache

## Hintergrund
Diese Studie untersucht die Fähigkeit von ChatGPT-4-Turbo,
wissenschaftsjournalistische Texte in einfache Sprache zu übersetzen und bewertet die Fehleranfälligkeit dieser Übersetzungen. 20 Artikel aus „Spektrum der Wissenschaft" wurden automatisch in einfache Sprache übersetzt.

Die durchschnittliche Artikellänge reduzierte sich dabei von
3.530 auf 2.140 Zeichen. Die Lesbarkeit der Original- und vereinfachten Texte wurde mittels der Wiener Sachtextformel verglichen, wobei die vereinfachten Texte deutlich verständlicher
waren. Eine inhaltsanalytische Untersuchung identifizierte im
Mittel 8,65 Fehler pro Text – hauptsächlich semantische sowie
einige syntaktische und morphologische Fehler. Typische Übersetzungsfehler umfassten Bedeutungsverschiebungen, Auslassungen wichtiger Informationen, Falschinformationen und fehlerhafte Umformungen direkter oder indirekter Rede. Empfehlungen der DIN 8581-1-Norm für einfache Sprache wurden
weitgehend eingehalten, Schwierigkeiten traten nur bei der Vermeidung von Konjunktiv, bildhafter Sprache und der durchgängigen Verwendung gleicher Begriffe auf. 

Die Ergebnisse zeigen, dass ChatGPT Texte vereinfachen kann,
jedoch besteht Verbesserungspotential bei der semantischen
Präzision. Zukünftige Untersuchungen sollen klären, wie die
Übersetzungsfehler das Textverständnis beeinflussen.

## Zitationsvorschlag
Hohenwalde, C.E., Wahl, M., & Lehmkuhl, M. (2024). Kann ChatGPT komplexe wissenschaftsjournalistische Texte
verständlich machen? Fehler bei der automatischen Übersetzung mit ChatGPT in einfache Sprache. 


## Skript

Mittels des folgenden Skripts lassen sich wissenschaftsjournalistische Texte mit ChatGPT in einfache Sprache übersetzen. Im Folgenden werden die hierzu notwendigen Schritte im Detail erklärt.

### 1. Voraussetzungen

In unserem Code benötigen wir einige Pakete. Falls diese noch nicht auf dem Rechner installiert sind, werden sie mit den angegebenen Befehlen nachinstalliert. 

In [ ]:
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install spacy_cleaner
!{sys.executable} -m pip install pyphen
!{sys.executable} -m pip install --upgrade langchain


### 2. Import

Sind alle Pythonpakete installiert, müssen wir diese importieren.

In [ ]:
import pandas as pd
import os

import string
import spacy
from spacy.lang.de.examples import sentences 
import pyphen

### 3. Definitionen für das Chatmodell

Um später auf ChatGPT zugreifen zu können, müssen einige grundlegende Definitionen für das Sprachmodell vorgenommen werden, z.B. der API-Key. Diesen findet man, wenn man eingeloggt ist, unter https://platform.openai.com/. Bitte füge den von "" umschlossenen String hinter das =-Zeichen ein.

In [ ]:
dic = pyphen.Pyphen(lang='de')

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI

openai_api_key = 
llm = ChatOpenAI(openai_api_key)

### 4. Textdatei laden

Im nächsten Schritt müssen wir die Textdatei laden, die vereinfacht werden soll. Bitte prüfe, ob der Pfad auch auf deinem Rechner korrekt ist. Falls dir bei der Textansicht durch print unerwartete Sonderzeichen im Text angezeigt werden, überprüfe das Encoding der Datei. 

In [ ]:
filename = "20_neutrinomasse.txt"
text_original = open("data/originaltexte/" + filename, "r", encoding='UTF-8').read()
print(text_original)

### 5. Definition der Prompts

Als nächstes definieren wir die Prompts, die zur Übersetzung der Texte in einfache Sprache genutzt werden. Diese orientieren sich an der DIN 8581-1-Norm für einfache Sprache. 


In [ ]:
chain_prompt_template_wichtige_infos = """
Du bist ein herausragender Wissenschaftsjournalist. 
Welche Ziele verfolgt der Autor mit diesem Text und was sind die für den Leser relevantesten Aussagen?
Halte dich kurz.
```
{TEXT}
```
"""

chain_prompt_template_textebene = """
Du bist ein Experte für einfache Sprache und ein herausragender Wissenschaftsjournalist.
Formuliere den Text so um, dass er einfach zu verstehen ist. 
Behalte den journalistischen Stil bei. 

Beachte insbesondere folgende Regeln: 

Textebene

1. Füge bei Bedarf zusätzliche Informationen und Beispiele ein.
Schlecht: Mobilität
Besser: Mobilität bedeutet, wie Menschen sich fortbewegen, zum Beispiel zu Fuß, mit dem Skateboard, Auto, Bus,
Zug oder Flugzeug.  

2. Nutze die direkte Anrede.
Schlecht: Der Antragsteller sollte das Dokument gründlich lesen.
Besser: Bitte lesen Sie das Dokument gründlich.

3. Vermeide Nominalstil. 
Schlecht: Von Seiten des Gremiums erfolgte keine Beteiligung an dieser Untersuchung.
Besser: Das Gremium beteiligte sich nicht an dieser Untersuchung.

4. Gliedere den Text in sinnvolle Abschnitte.

5. Verwende kurze Sätze.

6. Verwende für längere Aufzählungen nummerierte Listen.
Schlecht: Ins Freibad sollte man ein Handtuch, einen Badeanzug oder eine Badehose und Sonnencreme mitneh-
men.
Besser: Ins Freibad sollte man Folgendes mitnehmen:
— Handtuch;
— Badeanzug oder Badehose;
— Sonnencreme.

Beachte, dass folgende Informationen im finalen Text enthalten sein sollten:
```
{INFOS}
```

Text: 
```
{TEXT}
```
"""

chain_prompt_template_satzebene = """
Du bist ein Experte für einfache Sprache und ein herausragender Wissenschaftsjournalist.
Formuliere den Text so um, dass er einfach zu verstehen ist. 
Behalte den journalistischen Stil bei. 

Beachte insbesondere folgende Regeln: 

Satzebene

1. Nutze möglichst kurze Sätze. Kein Satz darf mehr als 15 Wörtern haben. Variiere die Satzlänge.

2. Benutze eine einfache Satzstruktur aus Subjekt – Prädikat – Objekt. Ein Satz darf maximal einen Nebensatz haben.
Schlecht: Der Zug kommt, wenn er keine Verspätung hat, um 12 Uhr an.
Besser: Der Zug kommt um 12 Uhr an, wenn er keine Verspätung hat.

3. Verbalklammer und Nominalklammer.
Zwischen den Teilen eines Prädikats und einem Artikel oder Pronomen und dem dazugehörige Substantiv sollten nicht zu viele Wörter stehen.
Schlecht: Irina hat Michael auf das Ende der Einreichungsfrist für den Antrag am 30. April hingewiesen.
Besser: Irina hat Michael gesagt, dass er den Antrag bis zum 30. April einreichen muss.

4. Benutze keine Verneinungen.
Schlecht: Sie haben uns die Angaben noch nicht geschickt. 
Besser: Bitte schicken Sie uns noch die fehlenden Angaben.

5. Texte sollen im Präsens geschrieben werden. Wenn Vergangenheitsformen notwendig sind, sollte das Perfekt genutzt werden.
Schlecht: Sie schrieben, lasen und beteten.
Besser: Sie haben geschrieben, gelesen und gebetet.

6. Vermeide Konjunktiv. Der Konjunktiv I darf nicht verwendet werden.
Schlecht: Anna sagte, sie wolle morgen kommen. Sie habe sich extra freigenommen.
Besser: Anna sagte, sie will morgen kommen. Sie sagte, dass sie sich extra freigenommen hat.

7. Ersetze Passivkonstruktionen durch Aktiv.
Schlecht: Es wurde eine Untersuchung seitens der Behörde durchgeführt.
Besser: Die Behörde führte eine Untersuchung durch.

Beachte, dass folgende Informationen im finalen Text enthalten sein sollten:
```
{INFOS}
```

Text: 
```
{TEXT}
```
"""

chain_prompt_template_wortebene = """
Du bist ein Experte für einfache Sprache und ein herausragender Wissenschaftsjournalist.
Formuliere den Text so um, dass er einfach zu verstehen ist. 
Behalte den journalistischen Stil bei. 

Beachte insbesondere folgende Regeln: 

Wortebene 

1. Benutze immer die gleichen Begriffe. Vermeide Synonyme.
Schlecht: Heute gehen alle Gläubigen in die Kirche, denn Ostern ist ein wichtiges Fest im Kirchenjahr. Im Gotteshaus singen die frommen Besucher Lieder.
Besser: Heute gehen alle Gläubigen in die Kirche, denn Ostern ist ein wichtiges Fest im Kirchenjahr. In der Kirche singen die Gläubigen Lieder.

2. Verwende konkrete und eindeutige Wörter. 
Schlecht: Werkzeug
Besser: Hammer

3. Benutze kurze und gebräuchliche Wörter.
Schlecht: Wägeeinheit
Besser: Waage

4. Vermeide Substantivierungen.
Schlecht: Das Öffnen der Verschlüsse ermöglicht die Wartung des Schmiernippels.
Besser: Öffnen Sie die Verschlüsse, um die Schmiernippel zu warten.

5. Vermeide Funktionsverbgefüge.
Schlecht: zur Sprache bringen
Besser: sagen

6. Vermeide Metaphern und Allegorien.
Schlecht: Eine Flut von Menschen strömt zum Stadion.
Besser: Sehr viele Menschen gehen zum Stadion.

7. Vermeide Fremdwörter. Wenn du trotzdem Fremdwörter benutzt, erkläre sie.
Schlecht: Evaluation
Besser: Auswertung

8. Löse Komposita auf. 
Schlecht: Infektionsschutzmaßnahmenverordnung
Besser: Verordnung über Maßnahmen zum Schutz vor Infektionen

Beachte, dass folgende Informationen im finalen Text enthalten sein sollten:
```
{INFOS}
```

Text: 
```
{TEXT}
```
"""

chain_prompt_template_finale_korrekturen = """
Du bist ein Experte für einfache Sprache und ein herausragender Wissenschaftsjournalist.
Formuliere den Text so um, dass er den Regeln der einfachen Sprache folgt und einfach zu verstehen ist. 
Behalte den journalistischen Stil bei. Beginne nicht mit "Liebe Leserinnen und Leser". Ende nicht mit einer Ansprache an die Leser oder einem Dank an die Leser.
Beachte, dass folgende Informationen im finalen Text enthalten sein sollten:
```
{INFOS}
```

Text: 
```
{TEXT}
```
"""

### 6. Text vereinfachen

Nun kann der Originaltext vereinfacht werden. Da verschiedene Vereinfachungsschritte nacheinander durchgeführt werden, kann es etwas dauern, bis alle Antworten von den OpenAI-Servern zurückgegeben werden.

In [ ]:
chain_prompt_wichtige_infos = PromptTemplate(template=chain_prompt_template_wichtige_infos, input_variables=["TEXT"])
chain_prompt_textebene = PromptTemplate(template=chain_prompt_template_textebene, input_variables=["INFOS" , "TEXT"])
chain_prompt_satzebene = PromptTemplate(template=chain_prompt_template_satzebene, input_variables=["INFOS" , "TEXT"])
chain_prompt_wortebene = PromptTemplate(template=chain_prompt_template_wortebene, input_variables=["INFOS" , "TEXT"])
chain_prompt_finale_korrekturen = PromptTemplate(template=chain_prompt_template_finale_korrekturen, input_variables=["INFOS" , "TEXT"])

chain_wichtige_infos = chain_prompt_wichtige_infos | ChatOpenAI(temperature=1, model="gpt-4-1106-preview", openai_api_key=openai_api_key) | StrOutputParser() 
infos = chain_wichtige_infos.invoke({"TEXT": text_original})
chain_textebene = chain_prompt_textebene | ChatOpenAI(temperature=1, model="gpt-4-1106-preview", openai_api_key=openai_api_key) | StrOutputParser() 
text_textebene = chain_textebene.invoke({"INFOS": infos, "TEXT": text_original})
chain_satzebene = chain_prompt_satzebene | ChatOpenAI(temperature=1, model="gpt-4-1106-preview", openai_api_key=openai_api_key) | StrOutputParser() 
text_satzebene = chain_satzebene.invoke({"INFOS": infos, "TEXT": text_textebene})
chain_wortebene = chain_prompt_wortebene | ChatOpenAI(temperature=1, model="gpt-4-1106-preview", openai_api_key=openai_api_key) | StrOutputParser() 
text_wortebene = chain_wortebene.invoke({"INFOS": infos, "TEXT": text_satzebene})
chain_finale_korrekturen = chain_prompt_finale_korrekturen | ChatOpenAI(temperature=1, model="gpt-4-1106-preview", openai_api_key=openai_api_key) | StrOutputParser() 
text_finale_korrekturen = chain_finale_korrekturen.invoke({"INFOS": infos, "TEXT": text_wortebene})

print(text_finale_korrekturen)

### 7. In einfache Sprache übersetzten Text speichern

Nachdem der Originaltext automatisch vereinfacht wurde, wird dieser im Ordner "vereinfachte_Texte" gespeichert. Bitte stelle sicher, dass dieser Ordner auf deinem Rechner angelegt sind.

In [ ]:
text_file = open("data/vereinfachte_Texte/" + filename, "w")
text_file.write(text_finale_korrekturen)
text_file.close()

### 8. Bewertung von Verständlichkeit: Wiener Sachtextformel 

Zuletzt können wir bestimmen, wie verständlich der Originaltext und der vereinfachte Text sind. Hierfür kann entweder die 1. Wiener Sachtextformel mit wiener_sachtext_formel() oder die 4. Wiener Sachtextformel mit wiener_sachtext_formel_4() genutzt werden.

Zur Interpretation: Je kleiner der Wert, desto verständlicher ist der Text.
- MS = Anteil der Wörter mit mehr als drei Silben
- SL = Mittlere Satzlänge (Anzahl der Wörter)
- IW = Anteil der Wörter mit mehr als 6 Buchstaben
- ES = Anteil der Einsilbigen Wörter

In [ ]:
def wiener_sachtext_formel(text):
    text_no_punct = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize Text
    nlp = spacy.load("de_core_news_sm")
    tokenized_text = nlp(text)
    tokenized_text_no_punct = nlp(text_no_punct)

    # Count sentences
    nr_sentences_text = 0
    for sent in tokenized_text.sents:
        nr_sentences_text += 1

    # Count long words and number of words  
    # Count words with more than 3 syllables and words with 1 syllable
    IW_long_words_text = 0
    nr_words_text =  0 
    nr_more_3_syl = 0
    nr_1_syl = 0
    for token in tokenized_text_no_punct:
        nr_words_text += 1
        if (len(token.text) > 6):
            IW_long_words_text += 1

        # Syllables
        sep = dic.inserted(str(token))
        syllables = len(sep.split("-"))
        if (syllables > 3):
            nr_more_3_syl += 1
        if (syllables == 1):
            nr_1_syl += 1   

    MS = nr_more_3_syl / nr_words_text
    SL = nr_words_text / nr_sentences_text
    IW = IW_long_words_text / nr_words_text
    ES = nr_1_syl / nr_words_text

    wiener_sachtext_1 = 0.1935 * MS + 0.1672 * SL + 0.1297 * IW - 0.0327 * ES - 0.875
    return(wiener_sachtext_1)

In [ ]:
def wiener_sachtext_formel_4(text):
    text_no_punct = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize Text
    nlp = spacy.load("de_core_news_sm")
    tokenized_text = nlp(text)
    tokenized_text_no_punct = nlp(text_no_punct)

    # Count sentences
    nr_sentences_text = 0
    for sent in tokenized_text.sents:
        nr_sentences_text += 1

    # Count long words and number of words  
    # Count words with more than 3 syllables and words with 1 syllable
    IW_long_words_text = 0
    nr_words_text =  0 
    nr_more_3_syl = 0
    nr_1_syl = 0
    for token in tokenized_text_no_punct:
        nr_words_text += 1
        if (len(token.text) > 6):
            IW_long_words_text += 1

        # Syllables
        sep = dic.inserted(str(token))
        syllables = len(sep.split("-"))
        if (syllables > 3):
            nr_more_3_syl += 1
        if (syllables == 1):
            nr_1_syl += 1   

    MS = nr_more_3_syl / nr_words_text
    SL = nr_words_text / nr_sentences_text
    IW = IW_long_words_text / nr_words_text
    ES = nr_1_syl / nr_words_text

    wiener_sachtext_4 = 0.2744 * MS + 0.2656 * SL - 1.693
    return(wiener_sachtext_4)

In [ ]:
text_original = open("data/originaltexte/" + filename, "r", encoding='UTF-8').read()
text_finale_korrekturen = open("data/vereinfachte_Texte/" + filename, "r").read()

print(wiener_sachtext_formel_4(text_original))
print(wiener_sachtext_formel_4(text_finale_korrekturen))